In [ ]:
!pip install confluent_kafka

In [ ]:
import csv
import json
from confluent_kafka import Producer, Consumer, KafkaError # to produce and consume data from Apache Kafka topics
import ssl

csv_file = 'Reviews.csv'

# SSL context configuration
# TODO: Load Apache Kafka certificates into certificates folder
apache_kafka_ssl_config = {
    'ssl.ca.location': 'certificates/ca.pem', 
    'ssl.certificate.location': 'certificates/service.cert',
    'ssl.key.location': 'certificates/service.key',
    'security.protocol': 'ssl',
}

apache_kafka_uri = ''  # TODO: Set URI for Apache Kafka

apache_kafka_input_topic_name = 'reviews'

producer = Producer({
    'bootstrap.servers': apache_kafka_uri, 
    **apache_kafka_ssl_config 
})

# Send a message to a Kafka topic
def send_message(message, topic_name):
    producer.produce(topic_name, json.dumps(message).encode('utf-8'))
    producer.flush()

# Read CSV file and send records to Kafka topic
with open(csv_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        # Convert each row to a JSON object
        record = {
            'Id': row['Id'],
            'ProductId': row['ProductId'],
            'UserId': row['UserId'],
            'ProfileName': row['ProfileName'],
            'HelpfulnessNumerator': row['HelpfulnessNumerator'],
            'HelpfulnessDenominator': row['HelpfulnessDenominator'],
            'Score': row['Score'],
            'Time': row['Time'],
            'Summary': row['Summary'],
            'Text': row['Text']
        }
        send_message(record, apache_kafka_input_topic_name)
        print(f"Sent record to Kafka topic: {record}")


